# Create a slice with PTP for precise time measurements

For running MF timestamp or OWL (one-way latency measurement) tools, slice nodes must meet prerequisites: 

+ Git and Dockerhub must be reachable 
+ Docker has to be running
+ PTP (Precision Time Protocol) clock must be running/

This notebook creates a 3-node slice and sets up all the the above

(Tested on 2023/08/18)

## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Exception: b'{\n    "errors": [\n        {\n            "details": "\\"invalid_token\\"\\nerror_description=\\"token not found\\"\\n",\n            "message": "Internal Server Error"\n        }\n    ],\n    "size": 1,\n    "status": 500,\n    "type": "error"\n}'


## Create an Experiment Slice

The following creates three nodes, on different sites, with basic NICs connected to FABRIC's FABnetv4 internet. Sites can be changed, but make sure that the site is PTP-compatible. 

Current list of PTP-compatible sites:

- STAR
- MAX
- MICH
- MASS 
- UTAH
- NCSA
- UCSD
- FIU
- CLEM
- CERN

In [6]:
slice_name = 'PTP_slice'

#[site1,site2,site3] = fablib.get_random_sites(count=2, 
#        avoid=["DALL","GPN","LBNL","RENC","SALT","TACC","UKY","WASH","NCSA","LOSA","GATECH","INDI","MAX", "MASS","NEWY","SRI","UCSD"])

# for faster execution, specify the sites (from the list above)
[site1, site2, site3] = ["STAR", "CERN", "CLEM"]

print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'


Sites: STAR, CERN, CLEM


In [7]:
slice_name = "PTP_slice"

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

Fail: name 'fablib' is not defined
<class 'slice'>


In [3]:

#Create Slice. add_fabnet() automatically adds an L3 interface on each node and assigns an IP address.
slice = fablib.new_slice(name=slice_name)

# Node1
node1 = slice.add_node(name=node1_name, site=site1, image='docker_rocky_8')
node1.add_fabnet()

# Node2
node2 = slice.add_node(name=node2_name, site=site2, image='docker_rocky_8')
node2.add_fabnet()

# Node3
node3 = slice.add_node(name=node3_name, site=site3, image='docker_rocky_8')
node3.add_fabnet()

#Submit Slice Request
slice.submit();


Retry: 9, Time: 280 sec


ID,064372be-d3f9-4a66-80a3-ee3672725f8e
Name,PTP_slice
Lease Expiration (UTC),2023-10-22 07:08:18 +0000
Lease Start (UTC),2023-10-21 07:08:19 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8f52c513-d45a-44dc-ba63-24a34b6e99a5,Node1,2,8,10,docker_rocky_8,qcow2,star-w1.fabric-testbed.net,STAR,rocky,2001:400:a100:3030:f816:3eff:fead:f377,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3030:f816:3eff:fead:f377,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c3296372-4c80-4b86-b317-1270174886fa,Node2,2,8,10,docker_rocky_8,qcow2,cern-w5.fabric-testbed.net,CERN,rocky,2001:400:a100:3090:f816:3eff:fe62:c516,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe62:c516,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
36498262-1a92-480b-b7ee-ffe1740bab86,Node3,2,8,10,docker_rocky_8,qcow2,clem-w1.fabric-testbed.net,CLEM,rocky,2620:103:a006:12:f816:3eff:fe4b:a047,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2620:103:a006:12:f816:3eff:fe4b:a047,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
de10b4f1-b3ed-4c7a-85fe-07714bace9b1,FABNET_IPv4_CERN,L3,FABNetv4,CERN,10.143.3.0/24,10.143.3.1,Active,
7914d022-b131-481d-98d3-452610f45ee9,FABNET_IPv4_CLEM,L3,FABNetv4,CLEM,10.136.5.0/24,10.136.5.1,Active,
b7cd438d-d7d5-4e7c-8eeb-0be6c1b6cc45,FABNET_IPv4_STAR,L3,FABNetv4,STAR,10.129.129.0/24,10.129.129.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-FABNET_IPv4_STAR_nic-p1,p1,Node1,FABNET_IPv4_STAR,100,auto,,02:5C:96:36:FB:07,enp7s0,enp7s0,10.129.129.2,6
Node2-FABNET_IPv4_CERN_nic-p1,p1,Node2,FABNET_IPv4_CERN,100,auto,,0E:AC:A4:66:B5:FC,enp7s0,enp7s0,10.143.3.2,4
Node3-FABNET_IPv4_CLEM_nic-p1,p1,Node3,FABNET_IPv4_CLEM,100,auto,,06:E6:7C:D5:FB:76,enp6s0,enp6s0,10.136.5.2,6



Time to print interfaces 280 seconds


## (Optional) Observe the Slice's Attributes

In [1]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

Exception: name 'fablib' is not defined


<!-- ## Configure IP Addresses

### Get the Assigned Subnet

FABnetv4 networks are assigned a subnet and gateway by FABRIC.  You can get the subnet and available IPs from the FABlib objects.  -->

## Check connectivity via Experimenter's network

In [4]:
try:
    node1 = slice.get_node(name=node1_name)   
    node3 = slice.get_node(name=node3_name)
    node3_addr = node3.get_interface(network_name=f'FABNET_IPv4_{node3.get_site()}').get_ip_addr()
    
    node1.execute('hostname')
    stdout, stderr = node1.execute(f'ping -c 5 {node3_addr}')  
    
except Exception as e:
    print(f"Exception: {e}")

Node1
PING 10.136.5.2 (10.136.5.2) 56(84) bytes of data.
64 bytes from 10.136.5.2: icmp_seq=1 ttl=61 time=32.3 ms
64 bytes from 10.136.5.2: icmp_seq=2 ttl=61 time=30.6 ms
64 bytes from 10.136.5.2: icmp_seq=3 ttl=61 time=30.6 ms
64 bytes from 10.136.5.2: icmp_seq=4 ttl=61 time=30.6 ms
64 bytes from 10.136.5.2: icmp_seq=5 ttl=61 time=30.5 ms

--- 10.136.5.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4006ms
rtt min/avg/max/mdev = 30.548/30.913/32.267/0.677 ms


# Prepare each node for time precision experiments

In [5]:
nodes = slice.get_nodes()

## Enable NAT64

### Upload and Execute the NAT64 Script

We will use a NAT64 script, which configures an *IPv6* node so that it can access non-*IPv6* services. To view the script, click [here](./nat64.sh). This script sets up the [Public NAT64 Service](https://nat64.net) on your node.

For more details on how uploading and executing scripts works on FABRIC, click [here](../upload_and_execute/upload_and_execute.ipynb).

Based on https://nat64.net/
Note that NAT64 could go away at any minute, as it is a public service ran independently by Kasper Dupont, and is not affiliated with FABRIC.

In [6]:
# Change this to all nodes

from ipaddress import ip_address, IPv6Address

for node in nodes:
    try:     
        # If the node is an IPv6 Node then configure NAT64
        if node.validIPAddress(node.get_management_ip()) == "IPv6":
            # needed to fix sudo unable to resolve error
            commands = """
            sudo echo -n "127.0.0.1 " | sudo cat - /etc/hostname  | sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7103       github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:8c52:7009       codeload.github.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:b9c7:6e85       objects.githubusercontent.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2600:1fa0:80b4:db49:34d9:6d1e::         ansible-galaxy.s3.amazonaws.com\n"|sudo tee -a /etc/hosts;
            sudo echo -n "2a01:4f9:c010:3f02:64:0:3455:9777       packages.confluent.io\n"|sudo tee -a /etc/hosts;
            """
            stdout, stderr = node.execute(commands, quiet=True)

        # Access non-IPv6 Services
        stdout, stderr = node.execute(f'sudo yum install -y -q git && git clone https://github.com/fabric-testbed/jupyter-examples.git')

        stdout, stderr = node.execute(f'ls jupyter-examples')

    except Exception as e:
        print(f"Exception: {e}")

 Cloning into 'jupyter-examples'...
 Changelog.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
LICENSE
Readme.md
requirements.txt
start_here.ipynb

Upgraded:
  git-2.39.3-1.el8_8.x86_64                git-core-2.39.3-1.el8_8.x86_64      
  git-core-doc-2.39.3-1.el8_8.noarch       perl-Git-2.39.3-1.el8_8.noarch      

 Failed to set locale, defaulting to C.UTF-8
  Cloning into 'jupyter-examples'...
 Changelog.md
LICENSE
Readme.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
requirements.txt
start_here.ipynb

Upgraded:
  git-2.39.3-1.el8_8.x86_64                git-core-2.39.3-1.el8_8.x86_64      
  git-core-doc-2.39.3-1.el8_8.noarch       perl-Git-2.39.3-1.el8_8.noarch      

 Failed to set locale, defaulting to C.UTF-8
  Cloning into 'jupyter-examples'...
 Changelog.md
LICENSE
Readme.md
configure.ipynb
docker_containers
fabric_examples
fabric_ssh_tunnel_tools
requirements.txt
start_here.ipynb


## Set up PTP (Precision Time Protocol)

#### Clone MeasurementFramework Git repo Locally

In [7]:
%%bash
rm -rf /tmp/MF;
git clone https://github.com/fabric-testbed/MeasurementFramework.git /tmp/MF;

Cloning into '/tmp/MF'...


#### Install and setup linux ptp package on all nodes 

In [8]:
pre_requisites = f"""
`sudo apt-get update;sudo apt-get -y install ansible git` || `sudo dnf -y install epel-release ;sudo dnf -y install ansible git`
"""
ansible_instructions = f"""
cd /tmp/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

for node in nodes:
    print (f"Installing PTP on {node.get_name()}")
    node.upload_directory('/tmp/MF/instrumentize/ptp/ansible','/tmp/')
    node.execute(f"{pre_requisites}"\
                 f"{ansible_instructions}"\
                )
    print (f"Installation of PTP Completed on {node.get_name()}\n\n")

Installing PTP on Node1

PLAY [all] *********************************************************************
 sudo: apt-get: command not found
sudo: apt-get: command not found
Importing GPG key 0xE451E5B5:
 Userid     : "CentOS Storage SIG (http://wiki.centos.org/SpecialInterestGroup/Storage) <security@centos.org>"
 Fingerprint: 7412 9C0B 173B 071A 3775 951A D4A2 E50B E451 E5B5
 From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-SIG-Storage
bash: line 1: Last: command not found
 
TASK [Gathering Facts] *********************************************************
ok: [127.0.0.1]

TASK [linuxptp : include] ******************************************************
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1
included: /tmp/ansible/roles/linuxptp/tasks/find_ptp_interfaces.yml for 127.0.0.1

TASK [linuxptp : Check if Interface enp7s0 is PTP capable] *********************
changed: [127

## Start Docker 

In [9]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker --help")

 Created symlink /etc/systemd/system/multi-user.target.wants/docker.service → /usr/lib/systemd/system/docker.service.
 
 Verify installtion... on Node1

Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Common Commands:
  run         Create and run a new container from an image
  exec        Execute a command in a running container
  ps          List containers
  build       Build an image from a Dockerfile
  pull        Download an image from a registry
  push        Upload an image to a registry
  images      List images
  login       Log in to a registry
  logout      Log out from a registry
  search      Search Docker Hub for images
  version     Show the Docker version information
  info        Display system-wide information

Management Commands:
  builder     Manage builds
  buildx*     Docker Buildx (Docker Inc., v0.10.5)
  checkpoint  Manage checkpoints
  compose*    Docker Compose (Docker Inc., v2.18.1)
  container   Manage containers
  context     M

# (Optional) Extend the slice (Add 14 days)

In [10]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

Lease End         : 2023-10-22 07:08:18 +0000


In [11]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

New lease end date : 2023-10-22 07:08:18 +0000


# Delete the Slice

In [ ]:
# try:
#     slice = fablib.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")

# Run OWL (One-Way Latency Measurements) 

Script for running OWL on one or more nodes in a slice. Output for each measurement is saved as a pcap file on the destination node. 
Current implementation on IPv4 experimenter interfaces only. Assumes each node has only 1 experimental interface.

(Tested on 2023/08/18)

# First things first

In [1]:
from fabrictestbed_extensions.fablib.fablib import fablib
import json

In [2]:
slice_name = "PTP_slice"

try:
    slice = fablib.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print (slice)

-----------  ------------------------------------
Slice Name   PTP_slice
Slice ID     064372be-d3f9-4a66-80a3-ee3672725f8e
Slice State  StableOK
Lease End    2023-11-04 07:19:47 +0000
-----------  ------------------------------------


In [3]:
from mflib import owl

# for debugging only
#import owl_local as owl

In [4]:
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'

try:    
    node1 = slice.get_node(name=node1_name)
    node2 = slice.get_node(name=node2_name)
    node3 = slice.get_node(name=node3_name)
    nodes = slice.get_nodes()
except Exception as e:
    print(f"Fail: {e}")   

## Look up IPs of nodes and decide which IP address to use 

Checking experiment IP addresses by filtering out meas_network addresses. Asssumes one experimenter interface per node.

In [5]:
node_ip_list = owl.nodes_ip_addrs(slice)
print(node_ip_list)

{'Node1': '10.129.129.2', 'Node2': '10.143.3.2', 'Node3': '10.136.5.2'}


## Check OWL prerequisites on all nodes in the slice

Runs the following test and prints output on each node:

+ `git clone`
+ `ps -ef | grep phc2sys`
+ `docker --help`

In [6]:
owl.check_owl_prerequisites(slice)


***** On Node1...

***** Is github reachable?
 Cloning into 'teaching-materials'...
 total 12
drwxrwxr-x. 6 rocky rocky 4096 Oct 21 07:14 jupyter-examples
drwxrwxr-x. 2 rocky rocky 4096 Oct 21 07:26 owl-output
drwxrwxr-x. 5 rocky rocky 4096 Oct 21 17:52 teaching-materials

***** Is PTP is enabled?
root       17036       1  0 07:18 ?        00:00:02 /usr/sbin/phc2sys -u 8 -E linreg -O +0 -s /dev/ptp1 -c enp7s0
root       17610       1  0 07:18 ?        00:00:00 /usr/sbin/phc2sys -u 8 -E linreg -O +0 -s /dev/ptp1 -c CLOCK_REALTIME
rocky      28596   28595  0 17:52 ?        00:00:00 bash -c ps -ef | grep phc2sys
rocky      28616   28596  0 17:52 ?        00:00:00 grep phc2sys

*****Is Docker installed?

Usage:  docker [OPTIONS] COMMAND

A self-sufficient runtime for containers

Common Commands:
  run         Create and run a new container from an image
  exec        Execute a command in a running container
  ps          List containers
  build       Build an image from a Dockerfile

****

## Pull OWL image from DockerHub

In [9]:
image_name="fabrictestbed/owl:0.1.4"

for node in nodes:
    node.execute(f"sudo docker pull {image_name}") 

0.1.4: Pulling from fabrictestbed/owl
Digest: sha256:7b50ef5ed9eb6c2fa293d85b842251370d12cdb759a18e52bbcff72905bae4f7
Status: Image is up to date for fabrictestbed/owl:0.1.4
docker.io/fabrictestbed/owl:0.1.4
0.1.4: Pulling from fabrictestbed/owl
Digest: sha256:7b50ef5ed9eb6c2fa293d85b842251370d12cdb759a18e52bbcff72905bae4f7
Status: Image is up to date for fabrictestbed/owl:0.1.4
docker.io/fabrictestbed/owl:0.1.4
0.1.4: Pulling from fabrictestbed/owl
Digest: sha256:7b50ef5ed9eb6c2fa293d85b842251370d12cdb759a18e52bbcff72905bae4f7
Status: Image is up to date for fabrictestbed/owl:0.1.4
docker.io/fabrictestbed/owl:0.1.4


# Run the experiment

There are 3 ways to use OWL

1. Specify each link with sender-capturer pair: this should be easiest method for most users.
2. Start each sender and capturer manually: this is useful when you want to have more precise control
3. Start on all links: this starts OWL on every pair of nodes in the slice (excluding meas-node if there is one)


**Note: By default, all the previously saved pcap files will be deleted when a node starts a new OWL capturer session. If this behavior is not desired, set `delete_previous_output` to `False`.**

## Method 1: Specify links by source and destination nodes
This examples runs OWL on the following links:

+ Node1 --> Node2
+ Node2 --> Node3
+ Node3 --> Node2

For each link, sender and capturer containers will start. If one node is a destination for more than 1 sender, only 1 capturer container instance will be created.

In [14]:
owl.start_owl(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 120, delete_previous_output=True)

Staring sender on Node1
a861868e1cdecc56664e542f0b0d57c5b66f7f04b07d96ad3b1547070f8ce4b4
Staring capturer on Node2
51778ae4cb5072717444676fcca34a531db4774db84cff9982b47029094adad2


In [15]:
owl.start_owl(slice, src_node=slice.get_node(name='Node2'), dst_node=slice.get_node(name='Node3'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

Staring sender on Node2
8b50d9b9a90dc803bf0d752ee2dd62ecb859ed9b1628e428cb8c5265e366cc2c
Staring capturer on Node3
6d37b68091bfaa515e1bf20b41ab100838ee2cb079ed11ac6ee493d05b0125ca


In [16]:
owl.start_owl(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, probe_freq=1, no_ptp=False, outfile=None, 
              duration = 600, delete_previous_output=True)

Staring sender on Node3
4a3bb71b0aece1e220d563d3e95a4da995eb572f8db1edfa271085556a12eed3
Staring capturer on Node2
owl-sender_10.143.3.2_10.136.5.2
owl-capturer_10.143.3.2
capturer already running on Node2


### Check if they are running

In [17]:
owl.check_owl_all(slice)

Node1
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
a861868e1cde   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   57 seconds ago   Up 57 seconds             owl-sender_10.129.129.2_10.143.3.2
Node2
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
8b50d9b9a90d   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   39 seconds ago   Up 39 seconds             owl-sender_10.143.3.2_10.136.5.2
51778ae4cb50   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   53 seconds ago   Up 52 seconds             owl-capturer_10.143.3.2
Node3
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
4a3bb71b0aec   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   34 seconds ago   Up 33 seconds             owl-sender_10.136.5.2_10.143.3.2
6d37b68091bf   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   37 seconds

### Stop all OWL containers

In [18]:
owl.stop_owl_all(slice)

Node1
a861868e1cde
Node2
8b50d9b9a90d
51778ae4cb50
Node3
4a3bb71b0aec
6d37b68091bf


### Verify

In [19]:
owl.check_owl_all(slice)

Node1
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Node2
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Node3
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


## Method 2: start each sender, caputrer separately

If user desires more granular control, sender and capturer can be started/stopped independently.


In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

In [ ]:
owl.start_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'), img_name=image_name, duration=120)

### Check how the containers are running

In [ ]:
owl.check_owl_all(slice)

### Stop OWL containers on each node

The method finds all containers with the prefix `owl-sender_` in their names, stops and removes them.

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node1'), dst_node=slice.get_node(name='Node3'))

In [ ]:
owl.stop_owl_capturer(slice,  dst_node=slice.get_node(name='Node2'))

In [ ]:
owl.stop_owl_sender(slice, src_node=slice.get_node(name='Node3'), dst_node=slice.get_node(name='Node2'))

### Verify

In [ ]:
owl.check_owl_all(slice)

## Method 3: start on all links

If OWL measurements are desired on ALL possible links, call the following method. It will start sender and capturer containers on all nodes with each node sending probe packets to all the rest of the nodes in the slice.

In [10]:
owl.start_owl_all(slice, img_name=image_name, probe_freq=1, outfile=None, duration=600, delete_previous=True)

Node1 --> Node2
Staring sender
5917d92c9bc18e5a11a5db957c6c753cd710fe0585b852993a3216a5bc371993
Staring capturer
11b4d7efd8b74a728d8e318cc1bd443c15c1352015c3e118cb261800cbba2a22
Node1 --> Node3
Staring sender
734b32f6e915bf1daf1f194d6e582bb7dec86b9d9d08944080573897487a433b
Staring capturer
96a5475b155071ff518841500c75473f57f12148874285ab81a737319ce2be19
Node2 --> Node1
Staring sender
3f1d1463af39756db1ca72bd93ceab412e09e5ed1f79815aa3238c7ac87700ab
Staring capturer
owl-sender_10.129.129.2_10.136.5.2
owl-sender_10.129.129.2_10.143.3.2
4f1ee652903f183ff49e9c51fa53d4d482600f7bfc4b10b6bd23de65a9e3838d
Node2 --> Node3
Staring sender
4f14e3b42fa7edfd354459f56a10e4aa7f7951d5ba282c42c6d78ffc3e5f7379
Staring capturer
owl-capturer_10.136.5.2
capturer already running on Node3
Node3 --> Node1
Staring sender
d485c5b72f968ba29e6c9d9b3e63d468a3dd10fedc1287ffa315519c61bb62d8
Staring capturer
owl-capturer_10.129.129.2
owl-sender_10.129.129.2_10.136.5.2
owl-sender_10.129.129.2_10.143.3.2
capturer already

### Check if they are running

In [11]:
owl.check_owl_all(slice)

Node1
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
4f1ee652903f   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   11 seconds ago   Up 11 seconds             owl-capturer_10.129.129.2
734b32f6e915   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   23 seconds ago   Up 23 seconds             owl-sender_10.129.129.2_10.136.5.2
5917d92c9bc1   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   32 seconds ago   Up 32 seconds             owl-sender_10.129.129.2_10.143.3.2
Node2
CONTAINER ID   IMAGE                     COMMAND                  CREATED          STATUS          PORTS     NAMES
4f14e3b42fa7   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   13 seconds ago   Up 13 seconds             owl-sender_10.143.3.2_10.136.5.2
3f1d1463af39   fabrictestbed/owl:0.1.4   "python3 sock_ops/ud…"   21 seconds ago   Up 20 seconds             owl-sender_10.143.3.2_10.129.129.2
11b4d7efd8b7   fabrictestbed/owl:0.1.4   "python3

### Stop all OWL containers

In [12]:
owl.stop_owl_all(slice)

Node1
4f1ee652903f
Node2
11b4d7efd8b7
Node3
96a5475b1550


### Verify

In [13]:
owl.check_owl_all(slice)

Node1
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Node2
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
Node3
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


# Get the Output files

By default, all the pcap files are saved in `/home/rocky/owl-output` as `{dest_ip}.pcap` on the desination node.

## Check the availability of output files

In [14]:
owl_output_dir = '/home/rocky/owl-output'

for node in nodes:
    print(node.get_name())
    node.execute(f"ls -lh {owl_output_dir}")

Node1
total 104K
-rw-r--r--. 1 root root 97K Oct 21 18:03 10.129.129.2.pcap
Node2
total 100K
-rw-r--r--. 1 root root 96K Oct 21 18:04 10.143.3.2.pcap
Node3
total 104K
-rw-r--r--. 1 root root 97K Oct 21 18:03 10.136.5.2.pcap


## Download the pcap files for analysis

In [15]:
for node in nodes:
    
    print(f"Downloading pcap files from {node.get_name()}")
    
    local_output_dir = '/home/fabric/work/my_notebooks/OWL_new/'
    owl.download_output(node, local_output_dir)

10.129.129.2.pcap
Downloaded /home/rocky/owl-output/10.129.129.2.pcap from Node1 to /home/fabric/work/my_notebooks/OWL_new/Node1/10.129.129.2.pcap
10.143.3.2.pcap
Downloaded /home/rocky/owl-output/10.143.3.2.pcap from Node2 to /home/fabric/work/my_notebooks/OWL_new/Node2/10.143.3.2.pcap
10.136.5.2.pcap
Downloaded /home/rocky/owl-output/10.136.5.2.pcap from Node3 to /home/fabric/work/my_notebooks/OWL_new/Node3/10.136.5.2.pcap
